<a href="https://colab.research.google.com/github/ThassiAmorim/Pdf2Excel/blob/main/pdf2excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF openpyxl pytesseract pillow
! apt install libtesseract-dev
!sudo apt install tesseract-ocr
!sudo apt-get install tesseract-ocr-por
import fitz  # PyMuPDF
import openpyxl
import pytesseract
from PIL import Image
import os
import cv2


# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Caminho para o arquivo PDF
pdf_path = '/content/drive/MyDrive/AutomatizaNotas/notas.pdf'

In [ ]:
from difflib import SequenceMatcher

def calculate_similarity(str1, str2):
    # Inicializar o objeto SequenceMatcher com as duas strings
    sequence_matcher = SequenceMatcher(None, str1, str2)

    # Calcular a similaridade como uma proporção entre as duas strings
    similarity_ratio = sequence_matcher.ratio()

    # Mapear a similaridade para uma escala de 0 a 10
    similarity_score = round(similarity_ratio * 10, 2)

    return similarity_score

# Função para extrair texto de uma imagem usando OCR
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    print(text)
    return text

# Função para extrair e converter imagens em texto
def extract_images_and_convert_to_text(page):
    text = ""
    images = page.get_images(full=True)

    for img_index, img in enumerate(images):
        xref = img[0]
        base_image = pdf_document.extract_image(xref)
        image_bytes = base_image["image"]

        # Salvar a imagem em um arquivo temporário para que o pytesseract possa lê-la
        image_path = f"temp_image_{img_index}.png"
        with open(image_path, "wb") as image_file:
            image_file.write(image_bytes)

        # Usar pytesseract para converter a imagem em texto
        extracted_text = pytesseract.image_to_string(image_path, lang='por')

        # Adicionar o texto extraído à saída
        text += f"Imagem {img_index + 1} (Página {page_num + 1}):\n{extracted_text}\n\n"


        # Remover o arquivo de imagem temporário
        image_file.close()

    return text

# Função para extrair dados do texto
def extract_data_from_text(text):
    lines =  [line.strip() for line in text.split('\n')]
    estabelecimento = ""
    total = ""

    for line in lines:
        print(f"-------------{line}---------------------")
        if "Estabelecimento:" in line:
            estabelecimento = line.split("Estabelecimento:")[1].strip()
            partes = estabelecimento.split("EF", 1)
            estabelecimento = partes[0]
        if calculate_similarity("VALOR TOTAL DA GUIA DE ENTREGA R$ ", line) > 8:
            try:
              total = line.split("R$ ")[1].strip()
            except:
              total = line.split("RS ")[1].strip()
            if total[len(total)-1] == ']' or total[len(total)-1] == ')':
              total = total[:-1]

            print(f"-----------{estabelecimento} ------------- {total}")

            return estabelecimento, total

# Abrir o PDF com PyMuPDF
pdf_document = fitz.open(pdf_path)

# Criar um novo arquivo Excel
excel_path = '/content/drive/MyDrive/AutomatizaNotas/extractions.xlsx'
workbook = openpyxl.Workbook()
worksheet = workbook.active
worksheet.title = "Extrações"
worksheet.append(["estabelecimento", "total"])

totals_dict = {}

for page_num in range(pdf_document.page_count):
    page = pdf_document[page_num]

    estabelecimento, total = extract_data_from_text(extract_images_and_convert_to_text(page))

    # Verificar se o estabelecimento já está no dicionário
    if estabelecimento in totals_dict:
        # Adicionar o valor ao total existente
        totals_dict[estabelecimento] += float(total)
    else:
        # Adicionar um novo par chave-valor ao dicionário
        totals_dict[estabelecimento] = float(total)

# Adicionar os dados do dicionário à planilha
for estabelecimento, total in totals_dict.items():
    worksheet.append([estabelecimento, total])

# Salvar o arquivo Excel
workbook.save(excel_path)

# Fechar o documento PDF
pdf_document.close()

print("Extração e criação do Excel concluídas!")